## Intro

In this PA, you will practice altering data frame objects in order to solve a riddle.

### Advice for troubleshooting your code

#### Check results often

If a chunk of code runs smoothly without giving you any error or warnings this does not necessarily mean it accomplished the desired task.

It is a good habit to check the results of your code every time you finish a task.

If you created or updated a data frame, make sure your edits did what you hoped.  Use the *Environment* or the `head()` function to investigate your changes.

#### Document your work

In the text before the code chunk, make sure to briefly state what the point of the chunk is.  This will remind later readers - which might be your future self! - what the desired output is.

Use comments in the code to give quick explanations of the goals of complex steps.

#### Two heads are better than one

It can be hard to spot bugs in code that you yourself wrote.t
(Have you ever sent a text message and not spotted an obvious  typo?  Same idea!)

Work with people around you if you can - if something goes wrong, ask a friend to take a peek at your code and see if any glaring errors (like *syntax error*) pop out.

#### Explain your code out loud

The best way to troubleshoot a sneaky bug is to explain out loud each step of your code, and what you hoped to accomplish.

If you are alone, try [Rubber Duck Debugging](https://en.wikipedia.org/wiki/Rubber_duck_debugging)!

#### Google is your friend

The whole of the internet is at your disposal!  Use it early, use it often.

Copy-paste the exact error message into Google. Chances are, somebody else had a similar problem and got a similar message.


Include package names in your search terms.  For example, "bar plot in plotnine with pandas data frame" is a better search than "bar plot in python".


## Part One: Data import and cleaning

This section will walk you through some steps to clean up today's dataset, so that you can use it more easily in Part Two.  

First, we declare our package dependencies and load the data.

(Note that the data loading function `read_csv` will give you an outpouring of helpful information about the dataset.  If you do not see the word "error", there is nothing to be concerned about.)


In [18]:
#libraries
import numpy as np
import pandas as pd

#reads in data
colleges = pd.read_csv("https://www.dropbox.com/s/bt5hvctdevhbq6j/colleges.csv?dl=1")

The code chunks in this section will cause an error and/or do the desired task incorrectly.  (Even the chunks that run without error are not correct!) You will need to find the mistake, and correct it, to complete the intended action.

None of the *text* is incorrect, only the *code* - your goal is to make the code do exactly what the documentation claims it will.

#### 1. Narrow down the columns to variables of interest

There are too many variables in this dataset.  We don't need all of them. We will limit our dataset down to only:

Name of the institution
City, State, and ZIP code of the institution
The Admissions Rate
The average SAT score
The number of undergraduate students
The in and out of state tuitions
Whether the school is public or private
The "REGION" variable.


In [19]:
#double brackets to store the list of variables wanted to keep
colleges_clean = colleges[['INSTNM', 'CITY', 'STABBR', 'ZIP', 'CONTROL', 'ADM_RATE', 'SAT_AVG', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS', 'REGION']]

#### 2. Remove observations that aren't needed

Drop the schools that are "private for-profit" (category 3).  We will not need these for the puzzle.


In [20]:
#stores all data that are only vals 1 and 2 for the control variable
colleges_clean = colleges_clean[(colleges_clean['CONTROL'] == 1) | (colleges_clean['CONTROL'] == 2)]

### 3. Adjust variable types: Numeric

Some of the columns of the dataset were automatically understood by the computer to be categorical, due to missing data appearing like strings.

Adjust the appropriate columns to be numeric.


In [21]:
#colleges_clean['TUITIONFEE_IN'] = colleges_clean['TUITIONFEE_IN'].astype('numeric')

#use pd.to_numeric to convert
colleges_clean['TUITIONFEE_IN'] = pd.to_numeric(colleges_clean['TUITIONFEE_IN'])

#colleges_clean['TUITIONFEE_OUT'] = colleges_clean['TUITIONFEE_OUT'].astype('numeric')
colleges_clean['TUITIONFEE_OUT'] = pd.to_numeric(colleges_clean['TUITIONFEE_OUT'])

#colleges_clean['SAT_AVG'] = colleges_clean['SAT_AVG'].astype('numeric')
colleges_clean['SAT_AVG'] = pd.to_numeric(colleges_clean['SAT_AVG'])

#colleges_clean['ADM_RATE'] = colleges_clean['ADM_Rate'].astype('numeric')
colleges_clean['ADM_RATE'] = pd.to_numeric(colleges_clean['ADM_RATE'])

### 4. Adjust variable types: Categorical

Similarly, columns of the dataset that are not numbers are automatically understood by the computer to be strings.

We know, however, that some variables are truly *categorical*, in that there are only a few categories (or "levels") that can be values of the column.

Adjust the appropriate columns to be understood as categorical.


In [22]:
#make sure to save the changes we are making
colleges_clean['CONTROL'] = colleges_clean['CONTROL'].astype('category')
colleges_clean['REGION'] = colleges_clean['REGION'].astype('category')

#### 5. Calculate new variables

Create a new variable called TUITION_DIFF which contains the difference between in and out of state costs.


In [23]:
#need to correctly refrence the variables wanted to calculate TUITON_DIFF
colleges_clean['TUITION_DIFF'] = colleges_clean['TUITIONFEE_OUT'] - colleges_clean['TUITIONFEE_IN']

#### 6. Deal with missing data

Drop all the rows with missing data.

(It is often a terrible idea to simply drop all rows with *any* missing data!  Usually, even if *some* of the information is missing, we don't want to throw out the entire row.  In this example, however, we'll be lazy.)


In [24]:
#axisa = 1 will drop columns with missing vals?. But we want to drop all rows! so do axis = 0
colleges_clean = colleges_clean.dropna(axis = 0)

## Part Two:  Identify the mystery college

Wow!  Your best friend Ephelia has been accepted to the college of her dreams!  Unfortunately, Ephelia is a very mysterious person, and she won't tell you directly which college this is.  You'll have to use her clues to figure out which school is her dream school.

Clues:

1. This college is located in Region 1.

2. This college's admission rate is in the first quartile for the region.

3. This college charges the same for in- and out-of-state tuition.

4. The average SAT score of this college is an odd number.

5. This college is NOT in New Hampshire or in the city of Boston.

6. More than 3,000 people apply to this college every year.  (Hint: Make a new variable to compute how many people apply.  Assume the size of an incoming class is 1/4 of the total undergraduate population, and remember that not everyone who *applies* is *admitted*.)

7. Ephelia is not going to an Ivy League school.

8. Of the two options remaining at this step, Ephelia will attend the cheaper one.


In [36]:
#clue 1
clue1 = colleges_clean[(colleges_clean["REGION"] == 1)]

#clue 2
#finds first quantile of admission rate
colleges_clean['ADM_RATE'].quantile(0.25)
clue2 = clue1[(clue1["ADM_RATE"] <= .53745)]

#clue 3
clue3 = clue2[(clue2["TUITION_DIFF"] == 0)]

#clue 4
clue4 = clue3[(clue3["SAT_AVG"] % 2 != 0)]

#clue 5
clue5 = clue4[(clue4['CITY'] != "New Hampshire") | (clue4['STABBR'] != "NH")]

#clue 6
#create applied variable
#first part is accepted freshmen 
#and then multiply that by total applied which would be 1 + 1-acceptance
clue5['APPLIED'] = .25 * clue5['UGDS'] * (1 + (1-clue5['ADM_RATE']))
#select the two that are over 3000
clue6 = clue5[(clue5["APPLIED"] > 3000)]

#clue7
clue6.head()
#not the ivy league (Harvard)

#So Ephilia is going to Boston University!

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF,APPLIED
1519,Boston University,Boston,MA,2215,2,0.2511,1401.0,52082.0,52082.0,16716.0,1,0.0,7308.6531
1549,Harvard University,Cambridge,MA,2138,2,0.0516,1523.0,48949.0,48949.0,7532.0,1,0.0,3668.8372
